# Generative Feedback Loops using Weaviate

In this notebook, we will demonstrate the power of Generative Feedback Loops with the example of creating personalized ads for AirBNB listings.

In [36]:
%pip install -Uq -r requirements.txt


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Get the dataset of AirBNB listings:

In [2]:
!wget -q https://github.com/weaviate/Generative-Feedback-Loops/raw/main/AB_NYC_2019.csv

In [3]:
import pandas as pd
import numpy as np

# Read CSV file
csv_file = "AB_NYC_2019.csv"
df = pd.read_csv(csv_file)

df.replace([np.inf, -np.inf, np.nan], None, inplace=True)

# Convert DataFrame to a list of dictionaries
data_list = df.to_dict(orient="records")

Now, set the environment variables for your Weaviate DB instance and your OpenAI API key.

In [4]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [5]:
import weaviate
from weaviate.classes.config import Property, DataType
import os

cluster_url = os.getenv("WCS_URL")
if not cluster_url:
    raise Exception("Please set WCS_URL environment variable")
wcs_api_key = os.getenv("WCS_API_KEY")
if not wcs_api_key:
    raise Exception("Please set WCS_API_KEY environment variable")
openai_key = os.getenv("OPENAI_API_KEY")
if not openai_key:
    raise Exception("Please set OPENAI_API_KEY environment variable")


client = weaviate.connect_to_wcs(
    cluster_url=cluster_url,
    auth_credentials=weaviate.auth.AuthApiKey(wcs_api_key),
    headers={
        "X-OpenAI-Api-Key": openai_key  # Replace with your API key
    },
)

Uncomment this line in case you want to delete an existing collection when starting over:

In [7]:
# client.collections.delete("AirBNB_Listings")

Now, we create a new collection for the AirBNB listings and define our schema:

In [8]:
import weaviate.classes as wvc

listings_collection = client.collections.create(
    name="AirBNB_Listings",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.config.Configure.Generative.openai(),
    properties=[
        wvc.config.Property(name="name", data_type=wvc.config.DataType.TEXT),
        wvc.config.Property(name="description", data_type=wvc.config.DataType.TEXT),
        wvc.config.Property(name="host_name", data_type=wvc.config.DataType.TEXT, skip_vectorization=True),
        wvc.config.Property(name="neighbourhood", data_type=wvc.config.DataType.TEXT, skip_vectorization=True),
        wvc.config.Property(
            name="neighbourhood_group", data_type=wvc.config.DataType.TEXT, skip_vectorization=True
        ),
        wvc.config.Property(name="price", data_type=wvc.config.DataType.TEXT, skip_vectorization=True),
    ],
)

Here, we upload the AirBNB listings data to Weaviate. Vectorization is done by Weaviate automatically:

In [9]:
limit = 10
with listings_collection.batch.dynamic() as batch:
    for data_obj in data_list[:limit]:
        data_properties = {}
        data_properties["name"] = data_obj["name"]
        data_properties["host_name"] = data_obj["host_name"]
        data_properties["neighbourhood"] = data_obj["neighbourhood"]
        data_properties["neighbourhood_group"] = data_obj["neighbourhood_group"]
        data_properties["price"] = str(data_obj["price"])
        
        batch.add_object(data_properties)

Using our prompt, Weaviate can generate a description for each listing in the collection:

In [10]:
generatePrompt = "Please write a description for the following AirBnB Listing in English. NAME: {name} HOST_NAME {host_name} NEIGHBOURHOOD {neighbourhood} NEIGHBOURHOOD_GROUP {neighbourhood_group} PRICE {price}. Please do not make up any information about the property in your description."

descriptions = listings_collection.generate.fetch_objects(single_prompt=generatePrompt)

Now we can add our generated descriptions as a property to each listing - our first generative feedback loop!

In [11]:
for description in descriptions.objects:
    print(description.generated)
    print(description.uuid)
    listings_collection.data.update(uuid=description.uuid, properties={
        'description': description.generated
    })

Welcome to The Village of Harlem in New York City! This cozy apartment is located in the vibrant neighborhood of Harlem in Manhattan. The space is hosted by Elisabeth and is perfect for those looking to experience the rich culture and history of this iconic area. With a price of $150 per night, this listing offers a comfortable and convenient stay in the heart of Harlem.
0cc0e7fa-f9e3-4534-b4f9-0b14ee6590f3
Welcome to our clean and quiet apartment home located in the charming neighborhood of Kensington in Brooklyn. Our cozy space is perfect for those looking for a peaceful retreat after a day of exploring the city. Just steps away from the park, you can enjoy a leisurely stroll or a picnic in the greenery. The apartment is hosted by John and is priced at $149 per night. We look forward to hosting you during your stay in Brooklyn!
1ca02e06-24bb-40e4-a765-e187583e59d4
Welcome to BlissArtsSpace! Located in the vibrant neighborhood of Bedford-Stuyvesant in Brooklyn, this cozy and artistic 

In [13]:
# client.collections.delete("Ads")

Now, we also want to generate ads. For this, we create a collection for ads and define the schema. Then, we add a reference property to our collection of listings, so each listing can have ads linked to it:

In [14]:
ads_collection = client.collections.create(
    name="Ads",
    properties=[
        wvc.config.Property(
            name="content",
            data_type=wvc.config.DataType.TEXT,
            description="The generated ad text",
        ),
        wvc.config.Property(
            name="target",
            data_type=wvc.config.DataType.TEXT,
            description="High-level audience target for this ad.",
        ),
    ],
)

listings_collection.config.add_reference(
    wvc.config.ReferenceProperty(name= "hasAd", target_collection="Ads")
)

Similarly to the previous example, we generate ads for each listing. We then create a new Ad object in the ads collection and link the corresponding listing to it:

In [16]:
generatePrompt = "Please write an engaging advertisement for the following AirBnb Listing: Description: {description} Please write the advertisement for this listing."

ads = listings_collection.generate.fetch_objects(single_prompt=generatePrompt)

for ad in ads.objects:
    print(ad.generated)
    uuid = ads_collection.data.insert(
        properties={
            "content": ad.generated,
        }
    )
    listings_collection.data.reference_add(
        from_uuid=ad.uuid, from_property="hasAd", to=uuid
    )

Experience the vibrant energy of Harlem in this cozy apartment hosted by Elisabeth! Immerse yourself in the rich culture and history of this iconic neighborhood, with easy access to all that Manhattan has to offer. For just $150 per night, you can enjoy a comfortable and convenient stay in the heart of Harlem. Book now and make unforgettable memories in The Village of Harlem!
Escape the hustle and bustle of the city and retreat to our cozy apartment in the charming neighborhood of Kensington, Brooklyn. Just steps away from the park, our clean and quiet space is the perfect oasis for your city getaway. Hosted by John, our apartment is priced at just $149 per night. Whether you're looking for a leisurely stroll in the greenery or a peaceful picnic, our apartment is the perfect home base for your Brooklyn adventure. Book now and experience the tranquility of Kensington during your stay in Brooklyn!
Escape to the artistic oasis of BlissArtsSpace in the heart of Brooklyn's Bedford-Stuyvesan

Next, we additionally want to generate ads that address a certain audience:

In [17]:
targets = ["young couples", "elderly couples", "single travelers"]

for target in targets:
    generatePrompt = "Please write an engaging advertisement for the following AirBnb Listing: Description: {description} Please write the advertisement for this listing to target "
    generatePrompt += target

    ads = listings_collection.generate.fetch_objects(single_prompt=generatePrompt)

    ads.objects[0].generated

    for ad in ads.objects:
        uuid = ads_collection.data.insert(
            properties={
                "content": ad.generated,
                "target": target,
            }
        )
    listings_collection.data.reference_add(from_uuid=ad.uuid, from_property="hasAd", to=uuid)

In [27]:
for ad in ads_collection.query.fetch_objects().objects:
    if ad.properties.get("target") is not None:
        print(ad.properties.get("content"))
        print(ad.properties.get("target"))

Escape to the bustling streets of Chinatown in Manhattan with a stay at our charming 1 bedroom apartment hosted by Ben. Perfect for elderly couples looking to explore the city, our cozy space offers all the comforts of home for just $150 per night. Immerse yourself in the vibrant culture of Chinatown while being just steps away from all the excitement of the Lower East Side. Book your stay with us today and experience the best of New York City!
elderly couples
Looking for a romantic getaway in the heart of Brooklyn? Look no further than our cozy apartment in the charming neighborhood of Kensington. Perfect for young couples looking for a peaceful retreat after a day of exploring the city, our clean and quiet space is just steps away from the park, perfect for a leisurely stroll or a romantic picnic.

Hosted by the friendly and welcoming John, our apartment is priced at just $149 per night, making it the perfect affordable option for your stay in Brooklyn. Don't miss out on this opportu

Next, we want to make our ads even more personalized. For this, we create a collection of individual users:

In [29]:
# client.collections.delete("Users")

In [30]:
users_collection = client.collections.create(
    name="Users",
    properties=[
        wvc.config.Property(
            name="name",
            data_type=wvc.config.DataType.TEXT,
        ),
        wvc.config.Property(
            name="biography",
            data_type=wvc.config.DataType.TEXT,
        ),
    ],
)


user_properties = {
    "biography": "Connor often travels with a golden doodle named Bowen.",
    "name": "Connor",
}

users_collection.data.insert(user_properties)

user_properties = {
    "biography": "Bob is a prolific weightlifter who will get upset if he doesn't have a good gym to workout in.",
    "name": "Bob",
}

users_collection.data.insert(user_properties)

UUID('ba5b14d6-f449-4de1-bcb0-6bc7f0a67f61')

To represent that an ad can be targeted to a certain user, we add a reference property to the ads collection:

In [31]:
ads_collection.config.add_reference(
    wvc.config.ReferenceProperty(
        name="hasUserTarget",
        target_collection="Users"
    )
)

Now, we generate personalized ads for each user and link the generated personalized ads to the corresponding user:

In [32]:
users = users_collection.query.fetch_objects()

for user in users.objects:
    generatePrompt = "Please write an engaging advertisement for the following AirBnb Listing: Description: {description} Please write the advertisement for this listing to target the following user:"
    generatePrompt += str(user.properties.get("biography"))

    ads = listings_collection.generate.fetch_objects(single_prompt=generatePrompt)

    for ad in ads.objects:
        uuid = ads_collection.data.insert(
            properties={
                "content": ad.generated,
            }
        )
        ads_collection.data.reference_add(from_uuid=uuid, from_property="hasUserTarget", to=user.uuid)
        listings_collection.data.reference_add(from_uuid=ad.uuid, from_property="hasAd", to=uuid)
        

In [33]:
results = ads_collection.query.fetch_objects(filters=wvc.query.Filter.by_ref_count("hasUserTarget").greater_or_equal(1))

In [34]:
for result in results.objects:
    print(result.properties.get("content"))

Attention all pet-loving travelers! Are you looking for the perfect place to stay in Harlem, New York City with your furry friend? Look no further than this cozy apartment hosted by Elisabeth. For just $150 per night, you and your golden doodle, Bowen, can experience the vibrant culture and history of Harlem while staying in a comfortable and pet-friendly space. Book now and make unforgettable memories in the heart of Harlem with your four-legged companion!
Attention all dog-loving travelers! Looking for a cozy and pet-friendly retreat in Brooklyn? Look no further than our clean and quiet apartment in the charming neighborhood of Kensington. Just steps away from the park, your furry friend Bowen will love exploring the greenery while you relax in our peaceful space. Hosted by John, our apartment is priced at $149 per night and is the perfect spot for you and your four-legged companion to unwind after a day of city adventures. Book now and make unforgettable memories with Bowen in Brook